## Issues with the previous word lexicon

**Too slow**
- The previous word lexicon is a python list using the "index" function to locate words, hence very inefficient when it comes to locating each word, being a lot worse when having to do it multiple times in order to cater to adjectives, verbs, nouns and adverbs.
- In order to have better implementation, we will be using both hash tables and tries, and check their memory consumption as well

#### Checking the previous implementation:

In [1]:
from previousimplementation import wordlexicon as oldWL
import time
import sys

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def get_time(func, *args):
    start_time = time.time()
    func(*args)
    end_time = time.time()
    print(f"Process took: {(end_time-start_time)*1000} ms")

In [3]:
get_time(print, "hello")

hello
Process took: 0.019073486328125 ms


In [4]:
lexicon_list = oldWL.lexicon_list

In [5]:
print(lexicon_list[:10])
print(lexicon_list[400:410])
print(lexicon_list[240000:240010])
print(len(lexicon_list))

['that', 'an', 'most', 'don', 'now', 'with', 'where', 'yourselves', 'how', 'few']
['crested', 'openhanded', 'pursy', 'pulley', 'unbelt', 'dismantle', 'idolater', 'immunocompetence', 'iconolatry', 'disappoint']
['microprobe', 'elaliite', 'elkinstantonite', 'tschauner', 'phosphides', 'gey', 'wdbj', 'whiteboyd', 'bechtel', 'skloot']
240956


In [6]:
for word in ["microprobe", "thought", "some"]:
    get_time(lexicon_list.index, word)

Process took: 2.5370121002197266 ms
Process took: 0.5304813385009766 ms
Process took: 0.0016689300537109375 ms


**Clearly a linear time can be noticed from over here**

In [7]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [8]:
for word in ["thought", "think"]:
    for type in ["v", "n", "r", "a"]:
        print(f"Checking time constraint with word: {word} and type: {type}")
        lemmatizer.lemmatize(word, type)

Checking time constraint with word: thought and type: v
Checking time constraint with word: thought and type: n
Checking time constraint with word: thought and type: r
Checking time constraint with word: thought and type: a
Checking time constraint with word: think and type: v
Checking time constraint with word: think and type: n
Checking time constraint with word: think and type: r
Checking time constraint with word: think and type: a


The lemmatizer does not take a lot of time, hence we can use it in our wordlexicon

In [9]:
lexicon_pb = oldWL.lexicon

In [10]:
print(sys.getsizeof(lexicon_list))
print(sys.getsizeof(lexicon_pb))

1927704
80


A python lexicon list takes about 2 MBs in memory 

**Hence, currently memory is not an issue, but time efficiency is**

We will implement two data structures to deal with this: 
- Hash Tables
- A Trie

In [11]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False
        self.index = -1

class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.size = 0

    def insert(self, word, index):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True
        node.index=index
        self.size+=1

    def get_index(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return -1
            node = node.children[char]
        return node.index

In [12]:
demoTrie = Trie()

In [13]:
demoTrie.insert("word", 3)
demoTrie.insert("wording", 4)
demoTrie.get_index("word")

3

In [20]:

punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [14]:
lexiconTrie = Trie()
for index, word in enumerate(lexicon_list):
    lexiconTrie.insert(word, index)

In [15]:
for word in ["microprobe", "thought", "some"]:
    get_time(lexicon_list.index, word)

Process took: 2.8772354125976562 ms
Process took: 0.8378028869628906 ms
Process took: 0.0019073486328125 ms


In [21]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
"microp*robe".translate(str.maketrans('', '', string.punctuation))

'microprobe'

In [27]:
removed_punctuations = [word.translate(str.maketrans('', '', string.punctuation)) for word in lexicon_list]

In [28]:
len(removed_punctuations)

240956

In [29]:
len(list(set(removed_punctuations)))

236102

In [16]:
for word in ["microprobe", "thought", "some"]:
    get_time(lexiconTrie.get_index, word)

Process took: 0.00667572021484375 ms
Process took: 0.00286102294921875 ms
Process took: 0.0019073486328125 ms


**Huge huge win :)**

In [17]:
sys.getsizeof(lexiconTrie)

48

In [18]:
import pickle

In [19]:
with open("lexiconTrie.pkl", "wb") as file:
    pickle.dump(lexiconTrie, file)

**It is 30Mbs now**

We will skip making the hashfunction coz boo

### Making a function to lemmatize each word and get the word ID afterwards
We will copy the previous function

In [76]:
%%time
with open("lexiconTrie.pkl", "rb") as file:
    lexTrie = pickle.load(file)

CPU times: user 4.59 s, sys: 26.2 ms, total: 4.62 s
Wall time: 4.58 s


In [81]:
%%time
for i in range(1000000):
    lexTrie.get_index("check")

CPU times: user 253 ms, sys: 0 ns, total: 253 ms
Wall time: 252 ms


In [3]:
from wordlexicon.trie import Trie, TrieNode

In [5]:
%%time
from wordlexicon.wordlexicon import GetLexiconSize, return_wordID, lexTrie

CPU times: user 13 µs, sys: 5 µs, total: 18 µs
Wall time: 23.4 µs


In [9]:
def get_word_from_key(root, key):
    current = root
    word = []

    while key:
        found = False
        for child in current.children.values():
            if child.is_end_of_word and key == child.index:
                word.append(chr(child.index + ord('a')))
                return ''.join(reversed(word))
            elif child.is_end_of_word and key > child.index:
                key -= child.index
                current = child
                found = True
                break
            elif not child.is_end_of_word:
                key -= 1

        if not found:
            return None

    return None

In [12]:
print(get_word_from_key(lexTrie.root, 6))

None


In [6]:
get_time(return_wordID, "runner")

Process took: 0.010728836059570312 ms


In [10]:
%%time
for word in ["each", "thing", "thought", "brothers", "brother", "good", "better", "best"]:
    print(return_wordID(word))

172
43379
48510
117019
117019
24242
99721
147351
CPU times: user 57 µs, sys: 32 µs, total: 89 µs
Wall time: 89.9 µs


In [8]:
%%time
return_wordID("brothers")

CPU times: user 19 µs, sys: 10 µs, total: 29 µs
Wall time: 31.5 µs


117019

In [12]:

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [14]:
lemmatizer.lemmatize("better", "a")

'good'

In [15]:
lex_list = [-1 for _ in range(GetLexiconSize())]

In [16]:
len(lex_list)

301647

In [2]:
from wordlexicon.trie import Trie, TrieNode
from wordlexicon.wordlexicon import GetLexiconSize, return_wordID, lexTrie

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
total_list = lexTrie.get_all_words()

In [11]:
lexTrie.root.children

{'t': <wordlexicon.trie.TrieNode at 0x7f51baf89210>,
 'a': <wordlexicon.trie.TrieNode at 0x7f51b6f82fe0>,
 'm': <wordlexicon.trie.TrieNode at 0x7f51b28069e0>,
 'd': <wordlexicon.trie.TrieNode at 0x7f51ae312d40>,
 'n': <wordlexicon.trie.TrieNode at 0x7f51aac1d900>,
 'w': <wordlexicon.trie.TrieNode at 0x7f51a861b040>,
 'y': <wordlexicon.trie.TrieNode at 0x7f51a6278d00>,
 'h': <wordlexicon.trie.TrieNode at 0x7f51a5ca73a0>,
 'f': <wordlexicon.trie.TrieNode at 0x7f51a2e58ca0>,
 'o': <wordlexicon.trie.TrieNode at 0x7f51ad640220>,
 'b': <wordlexicon.trie.TrieNode at 0x7f519cf4cca0>,
 's': <wordlexicon.trie.TrieNode at 0x7f5198e4e680>,
 'v': <wordlexicon.trie.TrieNode at 0x7f5191c47280>,
 'u': <wordlexicon.trie.TrieNode at 0x7f5190a481c0>,
 'i': <wordlexicon.trie.TrieNode at 0x7f518f37da80>,
 'c': <wordlexicon.trie.TrieNode at 0x7f518d8e53c0>,
 'l': <wordlexicon.trie.TrieNode at 0x7f5187c6c9a0>,
 'r': <wordlexicon.trie.TrieNode at 0x7f5185196f80>,
 'j': <wordlexicon.trie.TrieNode at 0x7f518038

In [13]:
total_list[50000:51000]

[('appmonitored', 240956),
 ('appdictated', 240956),
 ('appy', 240956),
 ('appétit', 240956),
 ('apphappy', 240956),
 ('appharvest', 259642),
 ('appplicator', 285030),
 ('apa', 179791),
 ('apatosaur', 8329),
 ('apatosaurus', 119484),
 ('apatosaurus_excelsus', 96972),
 ('apatow', 151119),
 ('apatowproduced', 240956),
 ('apatetic_coloration', 17849),
 ('apatura', 127874),
 ('apatura_iris', 66296),
 ('apathy', 74796),
 ('apathetic', 122350),
 ('apathetically', 133030),
 ('apatite', 90575),
 ('apatchenko', 240956),
 ('apata', 240956),
 ('apalachicola', 109645),
 ('apalachicola_rosemary', 64432),
 ('apalachicola_river', 138213),
 ('apac', 199835),
 ('apache', 97046),
 ('apache_dance', 68563),
 ('apache_devil_dance', 92366),
 ('apace', 125428),
 ('apact', 240956),
 ('apar', 120413),
 ('apart', 90536),
 ('apartment', 138440),
 ('apartment_house', 101174),
 ('apartment_building', 113410),
 ('apartmentfriendly', 184392),
 ('apartmentsharing', 204039),
 ('apartmentshouses', 226289),
 ('apartment

In [9]:
return_wordID("help")

92133

## There were some errors with the trie implementation so we will be using a dictionary instead

In [1]:
from wordlexicon.previousimplementation import wordlexicon as oldWL
import time
import sys
import string

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
lexicon_list = oldWL.lexicon_list

In [3]:
lexicon_list[:30]

['that',
 'an',
 'most',
 'don',
 'now',
 'with',
 'where',
 'yourselves',
 'how',
 'few',
 'did',
 'other',
 'ma',
 'than',
 'above',
 'be',
 'or',
 'haven',
 'wouldn',
 'am',
 'needn',
 'she',
 'aren',
 'their',
 'through',
 'by',
 'y',
 "mightn't",
 'these',
 'very']

In [4]:
removed_punctuations = [word.translate(str.maketrans('', '', string.punctuation)) for word in lexicon_list]

In [5]:
removed_punctuations = list(set(removed_punctuations))

In [9]:
len(removed_punctuations)

236102

In [10]:
word_index_dict = {word:index for index, word in enumerate(removed_punctuations)}

In [11]:
%%time
word_index_dict['these']

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 5.72 µs


23426

In [12]:
lexicon_path = "wordlexicon/lexicondict.pkl"

In [1]:
import wordlexicon.wordlexicon as wl

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


FileNotFoundError: [Errno 2] No such file or directory: 'wordlexicon/lexicondict.pkl'

In [2]:
from wordlexicon.lexiconDS import NewLexicon
newLex = NewLexicon

In [3]:
from wordlexicon.wordlexicon import newLex, return_wordID

In [25]:
len(removed_punctuations)

236102

In [26]:
newLex.worddict

{'tepidness': 0,
 '': 1,
 'bowse': 2,
 'yachting': 3,
 'zymosis': 4,
 'mangalyam': 5,
 'pretermcleveland': 6,
 'rassvet': 7,
 'chiropractor': 8,
 'serranos': 9,
 'propositionalcalculus': 10,
 'greatindiandesert': 11,
 'documentary': 12,
 'krasner': 13,
 'doledout': 14,
 'arizonan': 15,
 'epigastrium': 16,
 'abutter': 17,
 'guilttrip': 18,
 'worst': 19,
 'achaea': 20,
 'italianregion': 21,
 'tasmanian': 22,
 'honk': 23,
 'freeboarddeck': 24,
 'eurocentric': 25,
 'huron': 26,
 'youthfulsounding': 27,
 'internationalise': 28,
 'armenta': 29,
 'carbonated': 30,
 'prearrest': 31,
 'lensdirect': 32,
 'phylogeneticrelation': 33,
 'endtime': 34,
 'dhaba': 35,
 'directsupportingfire': 36,
 'prevaricate': 37,
 'demography': 38,
 'marrc': 39,
 'ontoxic': 40,
 'scarboroughwho': 41,
 'prebuttals': 42,
 'saimirisciureus': 43,
 'conviction': 44,
 'torre': 45,
 'norberg': 46,
 'kaepernicks': 47,
 'evert': 48,
 'chanar': 49,
 'sexuallytransmitteddisease': 50,
 'apodiformes': 51,
 'ragas': 52,
 'osx': 5

In [21]:
newLex.wordlist

['tepidness',
 '',
 'bowse',
 'yachting',
 'zymosis',
 'mangalyam',
 'pretermcleveland',
 'rassvet',
 'chiropractor',
 'serranos',
 'propositionalcalculus',
 'greatindiandesert',
 'documentary',
 'krasner',
 'doledout',
 'arizonan',
 'epigastrium',
 'abutter',
 'guilttrip',
 'worst',
 'achaea',
 'italianregion',
 'tasmanian',
 'honk',
 'freeboarddeck',
 'eurocentric',
 'huron',
 'youthfulsounding',
 'internationalise',
 'armenta',
 'carbonated',
 'prearrest',
 'lensdirect',
 'phylogeneticrelation',
 'endtime',
 'dhaba',
 'directsupportingfire',
 'prevaricate',
 'demography',
 'marrc',
 'ontoxic',
 'scarboroughwho',
 'prebuttals',
 'saimirisciureus',
 'conviction',
 'torre',
 'norberg',
 'kaepernicks',
 'evert',
 'chanar',
 'sexuallytransmitteddisease',
 'apodiformes',
 'ragas',
 'osx',
 'tbar',
 'modestlyapportioned',
 'jeremih',
 'ponzischeme',
 'chanceford',
 'durango',
 'spattered',
 'crisisstricken',
 'upturn',
 'south',
 'stokely',
 'prineville',
 'blueeyedmary',
 'familycyatheace

In [4]:
return_wordID("norberq")

norberq


TypeError: NewLexicon.get_wordID() missing 1 required positional argument: 'word'